In [6]:
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import mani_skill.envs
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from mani_skill.utils import gym_utils
from mani_skill.utils.wrappers.record import RecordEpisode
from mani_skill.vector.wrappers.sb3 import ManiSkillSB3VectorEnv
import torch
import numpy as np

SEED = 17
NUM_ENVS = 8



In [15]:
env = gym.make("CartPole-v1", render_mode = "rgb_array")
states, _ = env.reset(seed = SEED, options = {"low": -0.1, "high": 0.1})
check_env(env, warn = True)

ppo_model = PPO(policy = "MlpPolicy", env = env, learning_rate = 3e-4, n_steps = 2048, batch_size = 64, n_epochs = 10, gamma = 0.99, gae_lambda = .95,
clip_range = .2, verbose = 1, seed = SEED)



Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
total_timesteps = 100000

ppo_model.learn(total_timesteps = total_timesteps, progress_bar = True)
ppo_model.save("ppo_cartpole")

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 175      |
|    ep_rew_mean     | 175      |
| time/              |          |
|    fps             | 941      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 204          |
|    ep_rew_mean          | 204          |
| time/                   |              |
|    fps                  | 749          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0092073465 |
|    clip_fraction        | 0.061        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.585       |
|    explained_variance   | 0.7057566    |
|    learning_rate        | 0.0003       |
|    loss                 | 11.3         |
|    n_updates            | 60           |
|    policy_gradient_loss | -0.00513     |
|    value_loss           | 36.7         |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 220         |
|    ep_rew_mean          | 220         |
| time/                   |             |
|    fps                  | 702         |
|    iterations           | 3           |
|    time_elapsed         | 8           |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.005809396 |
|    clip_fraction        | 0.0337      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.575      |
|    explained_variance   | 0.7294357   |
|    learning_rate        | 0.0003      |
|    loss                 | 10.2        |
|    n_updates            | 70          |
|    policy_gradient_loss | -0.00507    |
|    value_loss           | 43.1        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 228         |
|    ep_rew_mean          | 228         |
| time/                   |             |
|    fps                  | 682         |
|    iterations           | 4           |
|    time_elapsed         | 11          |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.004491276 |
|    clip_fraction        | 0.0319      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.566      |
|    explained_variance   | 0.6930959   |
|    learning_rate        | 0.0003      |
|    loss                 | 8.97        |
|    n_updates            | 80          |
|    policy_gradient_loss | -0.00378    |
|    value_loss           | 45.5        |
-----------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 500         |
|    ep_rew_mean          | 500         |
| time/                   |             |
|    fps                  | 632         |
|    iterations           | 45          |
|    time_elapsed         | 145         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.003385966 |
|    clip_fraction        | 0.0458      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.322      |
|    explained_variance   | -0.52983654 |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00456    |
|    n_updates            | 490         |
|    policy_gradient_loss | -0.00206    |
|    value_loss           | 5.21e-06    |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 500         |
|    ep_rew_mean          | 500         |
| time/                   |             |
|    fps                  | 632         |
|    iterations           | 46          |
|    time_elapsed         | 148         |
|    total_timesteps      | 94208       |
| train/                  |             |
|    approx_kl            | 0.004396654 |
|    clip_fraction        | 0.0404      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.319      |
|    explained_variance   | -0.53680384 |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0106     |
|    n_updates            | 500         |
|    policy_gradient_loss | -0.0016     |
|    value_loss           | 3.52e-06    |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 500          |
|    ep_rew_mean          | 500          |
| time/                   |              |
|    fps                  | 632          |
|    iterations           | 47           |
|    time_elapsed         | 152          |
|    total_timesteps      | 96256        |
| train/                  |              |
|    approx_kl            | 0.0016928999 |
|    clip_fraction        | 0.0196       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.318       |
|    explained_variance   | -0.1793971   |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00438     |
|    n_updates            | 510          |
|    policy_gradient_loss | -0.00119     |
|    value_loss           | 2.8e-06      |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 500          |
|    ep_rew_mean          | 500          |
| time/                   |              |
|    fps                  | 632          |
|    iterations           | 48           |
|    time_elapsed         | 155          |
|    total_timesteps      | 98304        |
| train/                  |              |
|    approx_kl            | 0.0038388374 |
|    clip_fraction        | 0.0534       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.322       |
|    explained_variance   | -0.4521613   |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00405     |
|    n_updates            | 520          |
|    policy_gradient_loss | -0.00245     |
|    value_loss           | 2.28e-06     |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 500          |
|    ep_rew_mean          | 500          |
| time/                   |              |
|    fps                  | 632          |
|    iterations           | 49           |
|    time_elapsed         | 158          |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0012079959 |
|    clip_fraction        | 0.0355       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.316       |
|    explained_variance   | -1.1099062   |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0156      |
|    n_updates            | 530          |
|    policy_gradient_loss | -0.00247     |
|    value_loss           | 1.77e-06     |
------------------------------------------


In [13]:
num_envs = 4
max_episode_steps = 75
model = PPO.load('ppo_cartpole')
eval_env = gym.make("CartPole-v1", render_mode = "rgb_array")
obs, _ = eval_env.reset()
success = []

for i in range(max_episode_steps):
    action, _states = model.predict(obs, deterministic=True)
    # obs, rewards, dones, _, info = eval_env.step(action)
    output = eval_env.step(action)
    reward = output[1]
    
    print(f"output {i}: {output}")
    print(f"Reward {i}: {reward}\n")

    success.append(reward)


success_rate = np.mean(np.array(success))

print(f"Success Rate: {success_rate}")

output 0: (array([-0.01819461, -0.21334863,  0.01467597,  0.29898977], dtype=float32), 1.0, False, False, {})
Reward 0: 1.0

output 1: (array([-0.02246159, -0.40867665,  0.02065576,  0.5962649 ], dtype=float32), 1.0, False, False, {})
Reward 1: 1.0

output 2: (array([-0.03063512, -0.6040815 ,  0.03258106,  0.895382  ], dtype=float32), 1.0, False, False, {})
Reward 2: 1.0

output 3: (array([-0.04271675, -0.79962975,  0.0504887 ,  1.1981257 ], dtype=float32), 1.0, False, False, {})
Reward 3: 1.0

output 4: (array([-0.05870935, -0.99536735,  0.07445122,  1.5061954 ], dtype=float32), 1.0, False, False, {})
Reward 4: 1.0

output 5: (array([-0.07861669, -1.1913092 ,  0.10457513,  1.8211623 ], dtype=float32), 1.0, False, False, {})
Reward 5: 1.0

output 6: (array([-0.10244288, -1.3874259 ,  0.14099838,  2.1444194 ], dtype=float32), 1.0, False, False, {})
Reward 6: 1.0

output 7: (array([-0.13019139, -1.5836293 ,  0.18388677,  2.4771202 ], dtype=float32), 1.0, False, False, {})
Reward 7: 1.0

